In [6]:
import sys
sys.path.append('..')
import torch
import numpy as np
import torch.nn as nn
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import torchvision.transforms as tf

from src.models.hrnet.model import HRNet
from src.models.swin.model import Swin
from src.models.train import Trainer
from src.features.segmentation.dataset import SegmentationDataset

In [2]:
#hrnet = HRNet()
swin = Swin()

/opt/homebrew/Caskroom/miniconda/base/envs/ctci/lib/python3.10/site-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3550.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [3]:
# for name, param in hrnet.named_parameters():
#     if param.requires_grad:
#         print(name) 

net.cls_head.weight
net.cls_head.bias
net.aux_head.0.weight
net.aux_head.0.bias
net.aux_head.1.weight
net.aux_head.1.bias
net.aux_head.3.weight
net.aux_head.3.bias


In [3]:
trainer = Trainer(
    model=swin,
    device="mps",
    epoch_num=2,
    optim_params={
        "lr": 10e-5,
    }
)

In [7]:
transform = tf.Resize((256,256))

In [8]:
train_dataset = SegmentationDataset(
    images_dir="../data/split/train/train_images/",
    masks_dir="../data/split/train/train_masks/",
    image_transform=transform,
    mask_transform=transform
)

val_dataset = SegmentationDataset(
    images_dir="../data/split/valid/valid_images/",
    masks_dir="../data/split/valid/valid_masks/",
    image_transform=transform,
    mask_transform=transform
)


train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=2, shuffle=False)


In [12]:
for i, t in train_dataloader:
    swin = swin.to('cpu')
    out = swin(i)
    print(out.shape)
    break

torch.Size([2, 1024])


In [10]:
trainer.train(
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader
)

  0%|          | 0/6807 [00:04<?, ?it/s]


RuntimeError: 0D or 1D target tensor expected, multi-target not supported